# Next Steps

1. Extract/organize line configuration and conduit configuration data from old_exhibit
2. Write pdf_comparison class
3. Write pdf_markup class


In [1]:
%pip install pdfminer.six
%pip install pdfplumber
%pip install numpy pandas
%pip install textdistance
%pip install regex
%pip install pyyaml

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install ipympl
%pip install reportlab>=3.6.2
%pip install PyPDF2
%pip install ocrmypdf
%pip install pdf2jpg
%pip install PyMuPDF

In [1]:
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# To extract text from tables in PDF
import pdfplumber
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Optional, Iterable, Dict, Tuple
import regex
from textdistance import hamming, jaro, levenshtein
import yaml
import itertools

from tempfile import TemporaryDirectory
import os
os.environ['USE_TORCH'] = '1'
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
from PIL import Image
from PyPDF2 import PdfMerger
from ocrmypdf.hocrtransform import HocrTransform
import fitz

In [142]:
class pdf_data():
    def __init__(self,   
                 orig_filepath: str | Path, 
                 ocr_filepath:  str | Path, 
                 config:        str | Path,
                 key_val_sep:   str = ':' ,) -> None:
        self.orig_filepath = orig_filepath
        self.ocr_filepath = ocr_filepath
        self.config = config  
        self.text_df = None  
        self.px_col_sep = 8
        self.px_word_sep = 2
        self.col_sep_str = ' | '
        self.key_val_sep = key_val_sep

    def combine_key_value_pairs_in_words_df(self, words_df):
        drop_idxs = list()
        for wIdx, word in words_df.iterrows():
            if 'source' in words_df.columns and any(words_df['source'] == 'ocr'):
                test = (word['text'][-1] == self.key_val_sep) and \
                       (wIdx != words_df.index[-1])  
            else:
                test = (word['text'][-1] == self.key_val_sep) and \
                       (wIdx != words_df.index[-1]) and \
                       (words_df.loc[wIdx+1, 'top'] == word['top'])  

            if test:
                drop_idxs.append(wIdx+1)
                words_df.loc[wIdx, 'text'] = f"{word['text']}{words_df.loc[wIdx+1, 'text']}"  
                words_df.loc[wIdx, 'right'] = words_df.loc[wIdx+1, 'right']       
        words_df = words_df.drop(drop_idxs).reset_index(drop=True)    

        return words_df

    def fill_implicit_keys(self, section_header, left_mult=2, right_mult=2):
        section_dict = self.sections[section_header]
        for sub_idx, subsection_bounds in section_dict['bounds'].iterrows():    
            mask = (self.text_df['norm_top'   ] > subsection_bounds['top'   ]) & \
                   (self.text_df['norm_bottom'] < subsection_bounds['bottom']) 
            subsect_df = self.text_df.loc[mask, :]

            is_ocr = True if any(subsect_df['source'] == 'ocr') else False
            if is_ocr:
                # page = pdfplumber.open(self.ocr_filepath ).pages[subsect_df.loc[subsect_df.index[0], 'page']]
                words_df = self.ocr_words_df.copy()
                words_mask = (words_df['norm_top'   ] > subsection_bounds['top'   ]) & \
                             (words_df['norm_bottom'] < subsection_bounds['bottom']) 
                words_df = words_df.loc[words_mask, :]
                merged_rows = [idx for idx, word in words_df.iterrows() if ' ' in word['text']]
                for idx in merged_rows:
                    words_df.loc[idx, 'text'] = words_df.loc[idx, 'text'].split(' ')[0] 
                words_df['top'] = round(words_df['top'], -1)
            else:
                page = pdfplumber.open(self.orig_filepath).pages[subsect_df.loc[subsect_df.index[0], 'page']]
                words_df = self.get_words_df(page, subsect_df['top'].min(), subsect_df['bottom'].max())
                words_df.loc[:, 'top'] = np.round(words_df['top']) 
            

            words_df.sort_values(by=['top', 'left'], ignore_index=True, inplace=True)   

            words_df = self.combine_key_value_pairs_in_words_df(words_df)

            split_lines     = list()
            split_lines_row = list()
            split_lines_col = list()
            drop_idxs = list()
            for rIdx, line in subsect_df.iterrows():
                text_by_col = line['text'].split(self.col_sep_str)
                split_lines.extend(text_by_col)
                split_lines_row.extend([rIdx for k in range(len(text_by_col))])
                split_lines_col.extend(list(range(len(text_by_col))))
                
                # combine words_df to match phrases in split_lines
                for tIdx, token in enumerate(text_by_col):
                    combine_idxs = [idx for idx, word in words_df.iterrows() 
                                    if  word['text'] in token
                                    and word['top']+0.5 >= line['top']
                                    and word['bottom'] <= line['bottom']+5]
                    tmp_df = words_df.loc[combine_idxs, :]
                    
                    first_word = token.split(' ')[0]
                    possible_start_idx = tmp_df.index[tmp_df['text'] == first_word]
                    for start_idx in possible_start_idx:
                        phrase_idxs = range(start_idx, start_idx+len(token.split(' ')))
                        if all([True if idx in tmp_df.index else False for idx in phrase_idxs]): 
                            phrase = ' '.join(tmp_df.loc[phrase_idxs, 'text'])
                            if phrase == token:
                                words_df.loc[start_idx, ['text', 'right']] = [phrase, tmp_df.loc[phrase_idxs[-1], 'right']]
                                drop_idxs.extend(phrase_idxs[1:])             
                                break 

            words_df = words_df.drop(drop_idxs).reset_index(drop=True)

            if not all([True if phrase==token else False for phrase, token in zip(split_lines, words_df['text'])]):
                raise ValueError('the phrases in split_lines do not match the corrected tokens in words_df')

            for token, rIdx, cIdx, (wIdx, word) in zip(split_lines, split_lines_row, split_lines_col, words_df.iterrows()):
                if self.key_val_sep in token:
                    continue
                token_bounds = word[['left', 'right']]
                col_mask = ((words_df['right' ]- token_bounds['left'] > -left_mult*self.px_col_sep) & \
                            (words_df['left' ] - token_bounds['left'] <=  0                )) | \
                           ((words_df['right'] - token_bounds['right'] < right_mult*self.px_col_sep) & \
                            (words_df['right'] - token_bounds['right'] >= 0                ))
                same_column_tokens = words_df.loc[col_mask, 'text']
                
                implicit_key = [item.split(self.key_val_sep)[0] for item in same_column_tokens.values if len(item.split(self.key_val_sep)) == 2]
                if len(implicit_key) >= 1:
                    original_text = self.text_df.loc[rIdx, 'text']
                    text_cols = original_text.split(self.col_sep_str)
                    text_cols[cIdx] = f'{implicit_key[0]}{self.key_val_sep}{text_cols[cIdx]}'
                    self.text_df.loc[rIdx, 'text'] = self.col_sep_str.join(text_cols)  

    def identify_line_merge_sets(self, text_df=None):

        if text_df is None:
            text_df = self.text_df
        
        is_ocr = True if any(text_df['source'] == 'ocr') else False

        merge_sets = list()
        for idx, line in text_df.iterrows():
            if 'using' in line.text:
                stop = []
            if is_ocr:
                same_line_overlap_thresh = 0.0048
                # # top of word begins above bottom of current word, by at least 0.005 (.5% of page), and starts below top of word  
                # merge_set = np.where((text_df['norm_top']    < line['norm_bottom']) &
                #                     (np.abs(text_df['norm_top'] - line['norm_bottom']) > same_line_overlap_thresh) &
                #                     (text_df['norm_top']   >= line['norm_top']   )  )[0]
                # words that are below current word and overlap by at least same_line_overlap_thresh, or above current word and overlap by at least same 
                merge_set = np.where(((text_df['norm_top'] - line['norm_bottom'] < -same_line_overlap_thresh) &
                                      (text_df['norm_top'] >= line['norm_top'])                           
                                      ) |
                                     ((text_df['norm_bottom'] - line['norm_top'] >  same_line_overlap_thresh) & 
                                      (text_df['norm_top'] <= line['norm_top'])))
            else:
                merge_set = np.where((text_df['norm_top']    < line['norm_bottom']) &
                                    (text_df['norm_top']    > line['norm_top']   )  )[0]
                
            merge_set = text_df.index[merge_set]
            if len(merge_set) > 0:
                merge_set = merge_set.to_list()
                merge_set = sorted(merge_set + [idx]) if idx not in merge_set else merge_set 
                same_merge_set   = any([True if m_set == merge_set else False for m_set in merge_sets])
                overlapping_sets = [set_idx for set_idx, m_set in enumerate(merge_sets) if any(i for i in m_set if i in merge_set)]
                if same_merge_set:
                    continue
                elif len(overlapping_sets) == 1:
                    merge_sets[overlapping_sets[0]] = np.unique(merge_sets[overlapping_sets[0]] + merge_set).tolist() 
                elif len(overlapping_sets) > 1:
                    print('Have not written code to manage more than one overlapping set when combining lines')
                else:
                    merge_sets.append(merge_set)
                    print(' '.join(text_df.loc[merge_set,['text','left']].sort_values(by='left').text))
        
        text_df['line_idx'] = np.full((text_df.shape[0],), -1, dtype=int)
        for msIdx, merge_set in enumerate(merge_sets):
            text_df.loc[merge_set, 'line_idx'] = np.repeat(msIdx, len(merge_set))

        return merge_sets, text_df
        
    def get_words_df(self, page, top, bottom):
        page_crop = page.within_bbox((         0, top, 
                                        page.width, bottom))  

        words = page_crop.extract_words()  
        words_dict = dict(text=[], left=[], right=[], top=[], bottom=[])
        for word in words:
            if word['text'] == '|':
                continue
            word['text'] = word['text'].lower().replace('|','')
            for dict_key, word_key in zip(['text', 'left', 'right', 'top', 'bottom'],
                                            ['text',   'x0',    'x1', 'top', 'bottom']):
                words_dict[dict_key].append(word[word_key])
        
        words_df = pd.DataFrame.from_dict(words_dict)
        words_df.sort_values(by='left', ignore_index=True, inplace=True)

        return words_df

    def identify_columns_from_words_df(self, words_df):
        col_id = []
        col_num = 0
        prev_w_info = None
        for w_idx, w_info in words_df.iterrows():
            if prev_w_info is not None: 
                if (w_info['left'] - prev_w_info['right'] > self.px_col_sep):
                    col_num += 1
                elif (w_info['left'] - prev_w_info['right'] < 0): 
                    w_info['right'] = max([prev_w_info['right'], w_info['right']])
                # FIXME may need to reinstate this condition for original text
                # elif (w_info['left'] - prev_w_info['right'] < 0): 
                #     w_info['right'] = prev_w_info['right'] 
            col_id.append(col_num)
            prev_w_info = w_info.copy()
        words_df['col_id'] = col_id
        if 'source' in words_df.columns and any(words_df['source'] == 'ocr'):
            words_df['top'] = round(words_df['top'], -1)
        words_df.sort_values(by=['col_id', 'top', 'left'], ignore_index=True, inplace=True)

        col_phrases = []
        for col_id in words_df['col_id'].unique():
            col_df = words_df.loc[words_df['col_id'] == col_id, :]
            col_phrases.append(' '.join(col_df['text']))

        return col_phrases 

    def organize_single_lines(self, merge_sets, ocr_pdf, orig_pdf, text_df=None):
        
        if text_df is None:
            text_df = self.text_df

        merge_list = list(itertools.chain.from_iterable(merge_sets))
        for idx, line in text_df.iterrows():
            if idx not in merge_list:   
                if line['source'] == 'ocr':
                    page = ocr_pdf.pages [line['page']]
                else:
                    page = orig_pdf.pages[line['page']]

                words_df = self.get_words_df(page, line['top'], line['bottom'])
                words_df.loc[:, 'top'] = np.round(words_df['top']) 

                words_df = self.combine_key_value_pairs_in_words_df(words_df)

                col_phrases = self.identify_columns_from_words_df(words_df)

                text_df.loc[idx, 'text'] = self.col_sep_str.join(col_phrases)
        return text_df

    def condense_merge_sets(self, text_df = None, ocr_pdf = None, orig_pdf = None):

        if text_df is None:
            text_df = self.text_df
        is_ocr = True if any(text_df['source'] == 'ocr') else False

        drop_idxs = list()
        for line_idx in text_df['line_idx'].unique():
            if line_idx == -1:
                continue
            line_df = text_df.loc[text_df['line_idx'] == line_idx, :]
            top     = line_df['top'].min()    
            bottom  = line_df['bottom'].max()

            if not is_ocr:
                page = orig_pdf.pages[line_df.loc[line_df.index[0], 'page']]
                words_df = self.get_words_df(page, top, bottom)
                col_phrases = self.identify_columns_from_words_df(words_df)
                # replace first line in merge set with merged text and position info, then store indices of 
                # remaining merge set to drop at end of combine method
                text_df.loc[line_df.index[0], 
                            ['text','bottom','left','right', 'norm_bottom','top','norm_top',]
                            ] = [self.col_sep_str.join(col_phrases),
                                line_df['bottom'].max(),
                                line_df['left'].min(),
                                line_df['right'].max(),
                                line_df['norm_bottom'].max(),
                                line_df['top'].min(),
                                line_df['norm_top'].min(),] 

            drop_idxs.extend(line_df.index[1:]) 

        text_df = text_df.drop(drop_idxs).reset_index(drop=True)
        return text_df


    def organize_text_lines_by_row_and_column(self, text_df=None):
        merge_sets, text_df = self.identify_line_merge_sets(text_df)
        orig_pdf = pdfplumber.open(self.orig_filepath) 
        text_df = self.organize_single_lines(merge_sets, None, orig_pdf, text_df)
        text_df = self.condense_merge_sets   (text_df, None, orig_pdf)
        return text_df
    
    def split_text_lines_with_pdfplumber(self, pdf, element, page_idx, text_dict,):
        did_split = False
        page = pdf.pages[page_idx]
        page_crop = page.within_bbox((         0, page.height - element.y1, 
                                      page.width, page.height - element.y0))  

        text = page_crop.extract_text_lines()
        
        if len(text) > 1:
            did_split = True
            for line in text:

                norm_top    = page_idx + (line['top'   ] / page.height)
                norm_bottom = page_idx + (line['bottom'] / page.height)
                if np.logical_not(np.any((np.isclose(self.text_df['norm_top'   ], norm_top   , atol=1e-2)) &
                                         (np.isclose(self.text_df['norm_bottom'], norm_bottom, atol=1e-2))  )):
                    text_dict['page'       ].append(page_idx)
                    text_dict['bottom'     ].append(line['bottom'])
                    text_dict['top'        ].append(line['top'])
                    text_dict['left'       ].append(line['x0'])
                    text_dict['right'      ].append(line['x1'])
                    text_dict['text'       ].append(line['text'].lower().replace('|', ''))
                    text_dict['norm_top'   ].append(norm_top)
                    text_dict['norm_bottom'].append(norm_bottom)
                    text_dict['source'     ].append('ocr')
                    
                    print(f'\nstoring text from OCR on page {page_idx}:\n"{line['text'].lower()}"')

        return did_split    

    def get_text_lines_from_original(self):
        pdf = pdfplumber.open(self.orig_filepath)
        text_dict = dict(text=[], source=[], page=[], top=[], bottom=[], left=[], right=[], norm_top=[], norm_bottom=[], confidence=[])
        for page_idx, page in enumerate(pdf.pages):
            text_lines = page.extract_text_lines()
            for line in text_lines:
                norm_top    = page_idx + (line['top'   ] / page.height)
                norm_bottom = page_idx + (line['bottom'] / page.height)

                text_dict['page'       ].append(page_idx)
                text_dict['bottom'     ].append(line['bottom'])
                text_dict['top'        ].append(line['top'])
                text_dict['left'       ].append(line['x0'])
                text_dict['right'      ].append(line['x1'])
                text_dict['text'       ].append(line['text'].lower().replace('|', ''))
                text_dict['norm_top'   ].append(norm_top)
                text_dict['norm_bottom'].append(norm_bottom)
                text_dict['source'     ].append('original')
                text_dict['confidence' ].append(1.0)
        
        text_df = pd.DataFrame(text_dict)
        text_df = self.organize_text_lines_by_row_and_column(text_df=text_df)

        if self.text_df is None:
            self.text_df = text_df
        elif len(text_df) > 0:
            self.add_new_data_to_text_df(text_df)

        self.text_df.sort_values(by='norm_top', inplace=True, ignore_index=True, ascending=True)

    def identify_columns_from_words_df_docTR(self, words_df):
        col_id = []
        col_num = 0
        prev_w_info = None
        for w_idx, w_info in words_df.iterrows():
            if prev_w_info is not None: 
                if (w_info['left'] - prev_w_info['right'] > self.px_col_sep):
                    col_num += 1
                elif (w_info['left'] - prev_w_info['right'] < 0):
                    w_info['right'] = prev_w_info['right'] 
            col_id.append(col_num)
            prev_w_info = w_info.copy()
        words_df['col_id'] = col_id
        words_df.sort_values(by=['col_id', 'top', 'left'], ignore_index=True, inplace=True)

        col_phrases = []
        for col_id in words_df['col_id'].unique():
            col_df = words_df.loc[words_df['col_id'] == col_id, :]
            col_phrases.append(' '.join(col_df['text']))

        return col_phrases 

    def organize_single_lines_docTR(self, text_df=None):
        if text_df is None:
            text_df = self.text_df
        
        drop_idxs = list()
        for line_idx in text_df['line_idx'].unique():
            line_df = text_df.loc[text_df['line_idx'] == line_idx, :]

            line_df.index.names=['original_index']
            line_df = line_df.sort_values(by='left', 
                                          ascending=True, 
                                          ignore_index=False, 
                                          inplace=False
                                          )
            line_df.reset_index(drop=False, inplace=True)

            drop_idxs.extend(line_df['original_index'].values[1:].tolist())

            if line_idx == 2:
                stop  = [] 
            line_df = self.combine_key_value_pairs_in_words_df(line_df)

            col_phrases = self.identify_columns_from_words_df(line_df.copy())

            text_df.loc[line_df.loc[0, 'original_index'], 'text'] = self.col_sep_str.join(col_phrases)

        self.ocr_words_df = text_df.copy()
        text_df = text_df.drop(drop_idxs).reset_index(drop=True)

        return text_df

    def combine_merge_sets_docTR(self, text_df=None):

        if text_df is None:
            text_df = self.text_df

        drop_idxs = list()
        for line_idx in text_df['line_idx'].unique():
            line_df = text_df.loc[text_df['line_idx'] == line_idx, :]

            # replace first line in merge set with merged text and position info, then store indices of 
            # remaining merge set to drop at end of combine method
            text_df.loc[line_df.index[0], 
                        ['bottom', 
                         'left', 
                         'right', 
                         'norm_bottom',
                         'top',
                         'norm_top',
                         ]] = [line_df['bottom'].max(),
                               line_df['left'].min(),
                               line_df['right'].max(),
                               line_df['norm_bottom'].max(),
                               line_df['top'].min(),
                               line_df['norm_top'].min(),] 

            drop_idxs.extend(line_df.index[1:])             

        text_df = text_df.drop(drop_idxs).reset_index(drop=True)

    def identify_line_merge_sets_docTR(self, text_df=None):
        
        if text_df is None:
            text_df = self.text_df
        
        same_line_overlap_thresh = 0.005

        merge_sets = list()
        for idx, line in text_df.iterrows():
            merge_set = np.where((text_df['norm_top']    < line['norm_bottom']) &
                                 (np.abs(text_df['norm_top'] - line['norm_bottom']) > same_line_overlap_thresh) &
                                 (text_df['norm_top']   >= line['norm_top']   )  )[0]
            merge_set = text_df.index[merge_set]
            if len(merge_set) > 0:
                merge_set = merge_set.to_list()
                merge_set = sorted(merge_set.append(idx)) if idx not in merge_set else merge_set 
                same_merge_set   = any([True if m_set == merge_set else False for m_set in merge_sets])
                overlapping_sets = [set_idx for set_idx, m_set in enumerate(merge_sets) if any(i for i in m_set if i in merge_set)]
                if same_merge_set:
                    continue
                elif len(overlapping_sets) == 1:
                    merge_sets[overlapping_sets[0]] = np.unique(merge_sets[overlapping_sets[0]] + merge_set).tolist() 
                elif len(overlapping_sets) > 1:
                    print('Have not written code to manage more than one overlapping set when combining lines')
                else:
                    merge_sets.append(merge_set)
                    print(' '.join(text_df.loc[merge_set,['text','left']].sort_values(by='left').text))
        text_df['line_idx'] = np.full((text_df.shape[0],), -1, dtype=int)
        for msIdx, merge_set in enumerate(merge_sets):
            text_df.loc[merge_set, 'line_idx'] = np.repeat(msIdx, len(merge_set))

        return text_df

    def organize_text_lines_by_row_and_column_docTR(self, text_df=None):
        # text_df = self.identify_line_merge_sets_docTR(text_df)
        merge_sets, text_df = self.identify_line_merge_sets(text_df)
        text_df = self.organize_single_lines_docTR(text_df) #TODO 1 Figure out why frequency is dropped to next line in "using unlicensed frequencies", and correct incorrect word ordering (see photo on phone)
        # self.ocr_words_df = text_df.copy()
        # text_df = self.condense_merge_sets(text_df, None, None)
        return text_df

    def add_new_data_to_text_df(self, tmp_text_df):
        drop_idxs =list()
        for idx, line in tmp_text_df.iterrows():
            match_score = np.array([levenshtein.normalized_similarity(line['text'], match_line['text'])
                                    if abs(line['norm_top'] - match_line['norm_top']) < 0.05
                                    else 0
                                    for rIdx, match_line in self.text_df.iterrows()])
            if any(match_score > 0.8):
                drop_idxs.append(idx)

        tmp_text_df = tmp_text_df.drop(drop_idxs)

        self.text_df = pd.concat((self.text_df, tmp_text_df), axis=0, ignore_index=True)


    def get_text_lines_from_ocr(self, pages: Optional[Iterable[int]] = None) -> pd.DataFrame:
        text_dict = dict(text=[], source=[], page=[], top=[], bottom=[], left=[], right=[], norm_top=[], norm_bottom=[], confidence=[])
        for page_idx, page in enumerate(self.ocr_text['pages']):
            for block in page['blocks']:
                for line in block['lines']:
                    for word in line['words']:
                        text_dict['page'       ].append(page_idx)
                        text_dict['bottom'     ].append(page['dimensions'][0] * word['geometry'][1][1])
                        text_dict['top'        ].append(page['dimensions'][0] * word['geometry'][0][1])
                        text_dict['left'       ].append(page['dimensions'][1] * word['geometry'][0][0])
                        text_dict['right'      ].append(page['dimensions'][1] * word['geometry'][1][0])
                        text_dict['text'       ].append(word['value'].lower())
                        text_dict['norm_top'   ].append(page_idx+word['geometry'][0][1])
                        text_dict['norm_bottom'].append(page_idx+word['geometry'][1][1])
                        text_dict['confidence' ].append(word['confidence'])
                        text_dict['source'     ].append('ocr') 
        ocr_text_df = pd.DataFrame(text_dict)
        ocr_text_df = ocr_text_df.sort_values(by=['norm_top', 'left'], ascending=True, ignore_index=True)

        ocr_text_df = self.organize_text_lines_by_row_and_column_docTR(text_df=ocr_text_df)        
        
        if self.text_df is None:
            self.text_df = ocr_text_df
        elif len(ocr_text_df) > 0:
            self.add_new_data_to_text_df(ocr_text_df)

        self.text_df.sort_values(by='norm_top', inplace=True, ignore_index=True, ascending=True)

    def get_text_lines_from_ocr_AFTER_TESSERACT(self, pages: Optional[Iterable[int]] = None) -> pd.DataFrame:
        text_dict = dict(text=[], source=[], page=[], top=[], bottom=[], left=[], right=[], norm_top=[], norm_bottom=[],)
        for page_idx, page in enumerate(extract_pages(self.ocr_filepath)):
            pdf = pdfplumber.open(self.ocr_filepath)
            for el_idx, element in enumerate(page):
                if isinstance(element, LTTextContainer):
                    did_split = self.split_text_lines_with_pdfplumber(pdf, element, page_idx, text_dict)
                    if not did_split:
                        top = page.height - element.y1
                        bottom = page.height - element.y0 
                        norm_top    = page_idx + top    / page.height
                        norm_bottom = page_idx + bottom / page.height
                        if np.logical_not(np.any((np.isclose(self.text_df['norm_top'   ], norm_top   , atol=1e-2)) &
                                                 (np.isclose(self.text_df['norm_bottom'], norm_bottom, atol=1e-2))  )):
                            text_dict['page'       ].append(page_idx)
                            text_dict['bottom'     ].append(bottom)
                            text_dict['top'        ].append(top)
                            text_dict['left'       ].append(element.x0)
                            text_dict['right'      ].append(element.x1)
                            text_dict['text'       ].append(element.get_text().lower().replace('\n', '').replace('|', ''))
                            text_dict['norm_top'   ].append(norm_top)
                            text_dict['norm_bottom'].append(norm_bottom)
                            text_dict['source'     ].append('ocr')
    
                            print(f'\nstoring text from OCR on page {page_idx}:\n"{element.get_text().lower().replace('\n', '')}"')

                elif isinstance(element, LTRect):
                    print(f'page = {page_idx}, ypos={element.y0}, type={type(element)}')
                else:
                    print(f'page = {page_idx}, ypos={element.y0}, type={type(element)}')

        if self.text_df is None:
            self.text_df = pd.DataFrame(text_dict)
        elif len(text_dict['text']) > 0:
            tmp_text_df  = pd.DataFrame(text_dict)
            self.text_df = pd.concat((self.text_df, tmp_text_df), axis=0, ignore_index=True)

        self.text_df.sort_values(by='norm_top', inplace=True, ignore_index=True, ascending=True)

    def get_section_headers(self):
        with open(self.config, 'r') as file:
            self.config_data = yaml.safe_load(file) 
        
        self.sections = dict()
        for section in self.config_data['sections']:
            self.sections[section['header']] = dict(bounds = pd.DataFrame(),
                                                    extract = section['extract'])
            for key in section.keys():
                if key in ['header', 'extract']: continue
                self.sections[section['header']][key] = section[key]

    def get_section_bounds(self):

        self.get_section_headers()

        # detect start and end of sections
        prev_section_info = dict(header=None, df_row=None)
        prev_page = 0
        for row_idx, text_data in self.text_df.iterrows():
            detected_section = None
            for header in self.sections.keys():
                match = regex.search(f'{header}{{s<=3,i<=3,d<=3}}', text_data['text'], regex.BESTMATCH)    
                if match is not None:
                    detected_section = header
                    break

            if text_data['page'] > prev_page or detected_section is not None:
                if prev_section_info['header'] is not None:   # store end of sections
                    prev_df_idx = self.sections[prev_section_info['header']]['bounds'].index[-1] 
                    prev_bottom = prev_page+0.9999 if (text_data['page'] > prev_page) else text_data['norm_top'] 
                    self.sections[prev_section_info['header']]['bounds'].loc[prev_df_idx, ['bottom',]] = [prev_bottom]
                # store start of sections
                top = text_data['page'] if (text_data['page'] > prev_page) else text_data['norm_top']
                tmp_df = pd.DataFrame(data    = zip([top], [np.nan], [False]),
                                      columns = ['top', 'bottom', 'data_extracted'],)
                header = detected_section
                self.sections[header]['bounds'] = pd.concat((self.sections[header]['bounds'], tmp_df), ignore_index=True)
                prev_section_info['header'] = header
                if text_data['page'] > prev_page:
                    prev_page+=1
            prev_section_info['df_row'] = row_idx
        
        # store end of final section
        prev_bottom = self.text_df.loc[prev_section_info['df_row'], 'norm_bottom']
        prev_df_idx = self.sections[prev_section_info['header']]['bounds'].index[-1] 
        self.sections[prev_section_info['header']]['bounds'].loc[prev_df_idx, ['bottom',]] = [np.ceil(prev_bottom),]
    
    def extract_table(self, subsection_bounds):  
        page_num = int(np.floor(subsection_bounds['top'])) 
        pdf = pdfplumber.open(self.orig_filepath)
        table_page = pdf.pages[page_num]
        table_crop = table_page.within_bbox((               0, (subsection_bounds['top'   ]-page_num)*table_page.height, 
                                             table_page.width, (subsection_bounds['bottom']-page_num)*table_page.height))
        table = table_crop.extract_table()
        # print((subsection_bounds['top'   ]-page_num)*table_page.height, (subsection_bounds['bottom']-page_num)*table_page.height, table)
        return table
    
    def extract_text_lines_from_original(self, subsection_bounds):
        page_num = int(np.floor(subsection_bounds['top'])) 
        pdf = pdfplumber.open(self.orig_filepath)
        page = pdf.pages[page_num]
        page_crop = page.within_bbox((         0, (subsection_bounds['top'   ]-page_num)*page.height, 
                                      page.width, (subsection_bounds['bottom']-page_num)*page.height))  

        text = page_crop.extract_text_lines()

        for line in text:
            line['norm_top']    = page_num + (line['top']    / page.height)
            line['norm_bottom'] = page_num + (line['bottom'] / page.height)
            line['page']        = page_num
            line['bottom']      = page.height - line['bottom']
            line['top']         = page.height - line['top']
            line['text']        = line['text'].lower().replace('|', '')

        return text  

    def table_to_df(self, 
                    table: List[List[str]], 
                    extract_params: str | Dict | List[Dict],
                    ) -> pd.DataFrame:
        if type(extract_params) == str and 'col' in extract_params.lower():
            info_keys = list()
            data = list()
            for row in table:
                iKey = row.pop(0)
                info_keys.append(iKey.replace('\n', ' '))
                data.append(row)
            df = pd.DataFrame(data=np.array(data).transpose(), columns=info_keys)

        elif type(extract_params) == str and 'row' in extract_params.lower():
            info_keys = table[0]
            data = table[1:]
            df = pd.DataFrame(data=data, columns=info_keys)

        else:
            print('There is no method implemented for converting data with this extraction method to a DataFrame')
            df = None
        
        return df 
 
    def extract_table_data(self):
        for section_header, section_dict in self.sections.items():
            for sub_idx, subsection_bounds in section_dict['bounds'].iterrows():
                if subsection_bounds['data_extracted']:
                    continue

                table = self.extract_table(subsection_bounds)
                if table is not None:
                    data_df = self.table_to_df(table, section_dict['extract'])
                    data_df = data_df.loc[:, [col for col in data_df.columns if col.lower() != section_header]]
                    if 'data' in section_dict.keys(): 
                        section_dict['data'] = pd.concat((section_dict['data'], data_df),
                                                         axis=0,
                                                         ignore_index=True)
                    else:
                        section_dict['data'] = data_df
                    
                    section_dict['bounds'].loc[sub_idx, 'data_extracted'] = True

    def align_data_to_existing_df(self,
                                  section_dict: Dict,
                                  subsect_df: pd.DataFrame,
                                 ):
        if type(section_dict['extract']) == str and 'col' in section_dict['extract'].lower():
            info_keys_to_match = list(section_dict['data'].columns)
            original_info_keys = list(section_dict['data'].columns)
            info_keys = list()
            data      = list()
            for text in subsect_df['text']:
                row = text.split(self.col_sep_str)
                iKey = row.pop(0)
                iKey = iKey.replace('\n', ' ')
                match_score = np.array([levenshtein.normalized_similarity(iKey.lower(), matchKey.lower()) for matchKey in info_keys_to_match])
                sorted_match_score, sorted_info_keys = zip(*sorted(zip(match_score, info_keys_to_match), reverse=True))
                if iKey == 'equipment tip':
                    stop = []
                try:
                    top_key_matches = sorted_info_keys[:3]
                    
                    #correct any instances in which OCR dropped the last word (probably b/c it was on a second line), causing poor matching
                    if (sorted_match_score[0] < 0.9 
                        and len(iKey.split(' ')) == len(top_key_matches[0].split(' ')) 
                        and any([len(iKey.split(' ')) < len(key.split(' ')) for key in top_key_matches])):
                        match_score = np.array([levenshtein.normalized_similarity(iKey.lower(), ' '.join(matchKey.lower().split(' ')[:-1])) for matchKey in info_keys_to_match])
            
                    matched_key = info_keys_to_match.pop(np.where(match_score == match_score.max())[0][0])
                    info_keys.append(matched_key)
                    data.append(row)
                    print(iKey, matched_key)
                except:
                    print(f'\nNo matched key: iKey={iKey}, text={text}')
            
            # Add dummy data for unmatched keys
            expected_num_items = pd.Series([len(d) for d in data]).mode()[0]
            for key in info_keys_to_match:
                info_keys.append(key)
                data.append(['data_not_found' for k in range(expected_num_items)])

            # expected_num_items = pd.Series([len(d) for d in data]).mode()[0]
            wrong_count_info = [(idx, len(d)) for idx, d in enumerate(data) if len(d) != expected_num_items]
            for idx, num_items in wrong_count_info:
                data[idx] = ['wrong_num_columns' for k in range(expected_num_items)]
                print(f'\n"{info_keys[idx]}" contained the wrong number of columns in the line.')

            correct_order = [np.where(np.array(original_info_keys) == key)[0][0] for key in info_keys]
            _, info_keys = zip(*sorted(zip(correct_order, info_keys)))
            _, data      = zip(*sorted(zip(correct_order, data))) 

            data_df = pd.DataFrame(data=np.array(data).transpose(), columns=info_keys)
            section_dict['data'] = pd.concat((section_dict['data'], data_df),
                                              axis=0,
                                              ignore_index=True)
        return

    def get_multilevel_key_value_pairs(self, section_dict, items):
        filling_subheader = False
        for item in items: 
            split_item = item.split(self.key_val_sep)
            if len(split_item) == 2:
                key, value = split_item
                if len(value) == 0:
                    value = None
                else:
                    value = value[1:]  if value[0]  == ' '  else value
                    value = value[:-1] if value[-1] == '\n' else value
                    
                if filling_subheader:
                    section_dict['data'][stored_key][key] = value
                    print(f'{stored_key} - {key}{self.key_val_sep} {value}')
                else:
                    section_dict['data'][key] = value     
                    print(f'{key}{self.key_val_sep} {value}')
            elif len(split_item) == 1:
                stored_key = split_item[0]
                section_dict['data'][stored_key] = dict()
                filling_subheader = True

    def extract_key_value_pairs(self, 
                                section_dict: Dict, 
                                subsect_df: pd.DataFrame, 
                                ) -> Dict:
        section_dict['data'] = dict()
        for l_idx, line in subsect_df.iterrows(): 
            items = line['text'].split(self.col_sep_str)
            key_value_pairs = [item for item in items if len(item.split(self.key_val_sep)) == 2]
            if len(key_value_pairs) == len(items):
                for item in key_value_pairs:
                    key, value = item.split(self.key_val_sep)
                    if len(value) == 0:
                        value = None
                    else:
                        value = value[1:]  if value[0]  == ' '  else value
                        value = value[:-1] if value[-1] == '\n' else value
                    section_dict['data'][key] = value     
                    print(f'{key}{self.key_val_sep} {value}') 
            else:
                if line['source'] == 'original':
                    pdf = pdfplumber.open(self.orig_filepath)
                elif line['source'] == 'ocr':
                    pdf = pdfplumber.open(self.ocr_filepath)
                self.get_multilevel_key_value_pairs(section_dict, items)

    def extract_text_data(self):
        for section_header, section_dict in self.sections.items():
            for sub_idx, subsection_bounds in section_dict['bounds'].iterrows():
                if subsection_bounds['data_extracted']:
                    continue

                mask = (self.text_df['norm_top'   ] > subsection_bounds['top'   ]) & \
                       (self.text_df['norm_bottom'] < subsection_bounds['bottom']) 
                subsect_df = self.text_df.loc[mask, :]
                
                if 'data' in section_dict.keys() and type(section_dict['data']) == pd.DataFrame:
                    self.align_data_to_existing_df(section_dict, subsect_df)
                else:
                    self.extract_key_value_pairs(section_dict, subsect_df)
                
                section_dict['bounds'].loc[sub_idx, 'data_extracted'] = True

    def print_text(self):
        max_x = 0
        for page_text in self.text_containers:
            for text_container in page_text:
                if text_container.x1 > max_x:
                    max_x = text_container.x1
        for page_num, page_text in enumerate(self.text_containers):
            print('###############################################')
            print(f'Page {page_num}')
            print('###############################################') 
            for text_container in page_text:  
                text = text_container.get_text()
                # print(f'{text_container.y0} to {text_container.y1}', text)
                print(text_container.y1, text_container.y0, text)

    def convert_pdf_page_to_image(self, fitz_doc, image_path, idx, zoom=4):
        mat = fitz.Matrix(zoom, zoom)
        page = fitz_doc.load_page(idx)
        pix = page.get_pixmap(matrix=mat)
        pix.save(image_path)

    def write_ocr_text_to_pdfa(self, pdf_outpath, hocr_path, image_path):
        hocr = HocrTransform(hocr_filename=hocr_path,
                            dpi=1000,)
        hocr.to_pdf(
                    out_filename=pdf_outpath,
                    image_filename=image_path,
                    )
        
    def write_hocr_xml_file(self, hocr_path, page_xml):
        with open(hocr_path, 'w') as f:
            f.write(page_xml[0].decode())

    def create_pdfa_with_ocr(self, output_dir, output_stem, ocr_xml, fitz_doc):
        merger = PdfMerger()
        with TemporaryDirectory(dir= Path(os.getcwd())) as tmpdir:
            tmppath = Path(tmpdir)
            for idx, page_xml in enumerate(ocr_xml): 
                hocr_path   = tmppath / f'{output_stem}_hocr_page{idx}.xml'
                image_path  = tmppath / f'{output_stem}_image_page{idx}.png'
                pdf_outpath = tmppath / f'{output_stem}_docTR_page{idx}.pdf'
                self.write_hocr_xml_file(hocr_path, page_xml)
                self.convert_pdf_page_to_image(fitz_doc, image_path, idx, zoom=4) 
                self.write_ocr_text_to_pdfa(pdf_outpath, hocr_path, image_path)
                merger.append(pdf_outpath)
            merger.write(output_dir / f'{output_stem}_docTR.pdf' )
            merger.close()

    def run_ocr(self, ocr_predictor):
        output_base_path = self.orig_filepath.parent / self.orig_filepath.stem
        doc_pages = DocumentFile.from_pdf(self.orig_filepath)
        fitz_doc = fitz.open(self.orig_filepath)
        ocr_text = ocr_predictor(doc_pages) 
        self.ocr_text = ocr_text.export()
        ocr_xml = ocr_text.export_as_xml()
        self.create_pdfa_with_ocr(output_dir  = self.orig_filepath.parent, 
                                  output_stem = self.orig_filepath.stem, 
                                  ocr_xml = ocr_xml, 
                                  fitz_doc = fitz_doc)   


class ATC_amendment(pdf_data):
    def __init__(self, 
                 orig_filepath: str | Path, 
                 ocr_filepath:  str | Path, 
                 config:        str | Path,
                 key_val_sep:   str = ':' ,) -> None:        
        super().__init__(orig_filepath, ocr_filepath, config, key_val_sep)

    def extract_line_config_data_from_cells_containing_all_key_value_pairs(self, 
                                                                           equipment_df, 
                                                                           configuration_col,
                                                                           data_dict, 
                                                                           data_keys,
                                                                           storage_key):
        line_config_df = equipment_df.loc[:, configuration_col]
        for equipIdx, cell in line_config_df.items():
            for dKey in data_keys:
                num_match = len(regex.findall(f'{dKey}{{e<=1}}', cell))
                
                if num_match > 0:
                    # move thru each match (there may be multiple line configs in single cell)
                    prev_search_end = 0
                    for idx in range(num_match):
                        match = regex.search(f'{dKey}{{e<=1}}', cell[prev_search_end:], pos=idx)
                        key_span = (match.span()[0] + prev_search_end, match.span()[1] + prev_search_end)
                        
                        # find the next key match in the string to know where the value for this key ends
                        possible_next_keys_pos = list()
                        for next_dKey in data_keys:
                            next_key_match = regex.search(f'{next_dKey}{{e<=1}}', cell[key_span[1]:], pos=0) 
                            if next_key_match is not None:
                                possible_next_keys_pos.append(next_key_match.span()[0])
                        # extract the data token from the string
                        if len(possible_next_keys_pos) == 0:
                            data_token = cell[key_span[0] : ]
                        else:
                            token_end = key_span[1] + min(possible_next_keys_pos) 
                            data_token = cell[key_span[0] : token_end]
                            prev_search_end = token_end
                        data_token = data_token[ :-1] if data_token[-1] == '\n' else data_token
                        data_token = data_token.replace('\n', '')
                        print(data_token)
                        val = data_token.split(self.key_val_sep)[1]
                        val = val[1:] if val[0] == ' ' else val

                        data_dict[str(idx)][dKey][equipIdx] = val

        for line_num, line_data in data_dict.items():
            if any([True if any([True if val is not None else False for val in data_list]) else False 
                    for tmp_key, data_list in line_data.items()]):
                for dKey, values in line_data.items():
                    key = f'{storage_key}_{line_num}_{dKey}'
                    equipment_df[key] = values 
        
    def extract_line_config_data_from_separated_cells(self, 
                                                      equipment_df, 
                                                      configuration_cols,
                                                      data_dict, 
                                                      data_keys,
                                                      storage_key): 
        line_config_df = equipment_df.loc[:, configuration_cols]
        for equipIdx, row in line_config_df.iterrows():
            if levenshtein.normalized_similarity(row['Line Configuration'].lower(), 'n/a') < 0.3:
                line_config = regex.split(r'\n|;| - |-', row['Line Configuration']) # TODO 1 collect data by splitting on ; and -, store in data_dict (see photo on phone)
                line_config  = [token.strip() for token in line_config if token != '']

                # FIXME May need to fix this fix if it doesn't apply to other versions of old exhibit. 
                combine_idxs = [(idx, idx+1) 
                                for idx, (token, next_token) 
                                in enumerate(zip(line_config[:-1], line_config[1:])) 
                                if token[-1]=='"' and next_token[0]=='(']  
                for cIdxs in combine_idxs:
                    line_config[cIdxs[0]] = ' '.join(line_config[cIdxs[0]:cIdxs[1]+1])
                    del line_config[cIdxs[1]]
                # fix end 
                
                nKeys = len(data_dict['0'].keys())
                if not len(line_config) % nKeys == 0:
                    raise ValueError("The number of tokens in line_config is not a multiple of the expected number of keys for storing data")
                
                equip_idx_list = [int(idx/nKeys) for idx in range(len(line_config))]
                for dKey, token, equipIdx in zip(data_dict['0'].keys(), line_config, equip_idx_list):
                    data_dict['0'][dKey][equipIdx] = token    

            else: # TODO 2 deal with conduit vs line, make sure all data is stored appropriately
                for dKey, col in zip(data_dict['0'].keys(), line_config_df.columns):
                    data_dict['0'][dKey][equipIdx] = row[col]

        for line_num, line_data in data_dict.items(): 
            if any([True if any([True if val is not None else False for val in data_list]) else False 
                    for tmp_key, data_list in line_data.items()]):
                storage_key_list = [storage_key if ]
                for dKey, values in line_data.items():
                    if levenshtein.normalized_similarity(line_data['Type'].lower(), 'conduit') > 0.5:
                        tmp_storage_key = 'conduit'
                #     else:
                # tmp_storage_key = storage_key
                #     key = f'{tmp_storage_key}_{line_num}_{dKey}'
                #     equipment_df[key] = values       
                

    def align_line_configuration_data(self, group_key, max_line_types=2, type_key=None):
        
        pdf_data_keys    = self.sections['equipment specifications'][f'{group_key} keys']
        equipment_df = self.sections['equipment specifications']['data']
        
        if 'separated' in group_key:
            storage_key = f"{type_key.split(' ')[0]}_config" 
            storage_data_keys = [self.sections['equipment specifications'][config_group_key] 
                                 for config_group_key in self.sections['equipment specifications'].keys() 
                                 if type_key in config_group_key][0]
            configuration_cols = list()
            for dKey in pdf_data_keys:
                config_col_match_scores = [(col, levenshtein.normalized_similarity(col, dKey)) for col in equipment_df.columns]
                scores = np.array([score for _, score in config_col_match_scores])
                config_col = config_col_match_scores[np.argmax(scores)][0]    
                configuration_cols.append(config_col)
        else:
            storage_key = f"{group_key.split(' ')[0]}_config"
            storage_data_keys = pdf_data_keys
            configuration_cols = [col for col in equipment_df if group_key in col.lower()]
            
        data_dict = dict()
        for idx in range(max_line_types): 
            data_dict[str(idx)] = dict()
            for dKey in storage_data_keys:
                data_dict[str(idx)][dKey] = [None for k in range(equipment_df.shape[0])]

        if 'separated' in group_key:
            self.extract_line_config_data_from_separated_cells(equipment_df,
                                                               configuration_cols,
                                                               data_dict,
                                                               pdf_data_keys,
                                                               storage_key)
        else:
            self.extract_line_config_data_from_cells_containing_all_key_value_pairs(equipment_df, 
                                                                                    configuration_cols[0],
                                                                                    data_dict, 
                                                                                    pdf_data_keys,
                                                                                    storage_key,)
           
    def get_exhibit_name(self):
        for text_container in self.text_containers[0]:
            text = text_container.get_text()
            if 'exhibit' in text.lower():
                pattern = regex.compile(r'^\s+')
                exhibit = pattern.sub('', text.lower().replace('exhibit', '').replace('\n', ''))
                self.exhibit = exhibit

SyntaxError: invalid syntax (315532376.py, line 932)

In [143]:
new_ocr_pdf_path  = Path('amendments/New_Exhibit_Redacted_docTR.pdf')
new_orig_pdf_path = Path('amendments/New_Exhibit_Redacted.pdf')
old_ocr_pdf_path  = Path('amendments/Old_Exhibit_Redacted_docTR.pdf')
old_orig_pdf_path = Path('amendments/Old_Exhibit_Redacted.pdf')

config_path = Path(r'C:\Users\Dalton\Documents\personal_records\apex_consulting\doctr_ocr\configs\atc_extra_info_config.yaml')

det_arch_options  = ['linknet_resnet18',
                     'linknet_resnet34',
                     'linknet_resnet50',
                     'db_resnet50',
                     'db_mobilenet_v3_large',
                     'fast_tiny',
                     'fast_small',
                     'fast_base',]

reco_arch_options = ['crnn_vgg16_bn',
                     'crnn_mobilenet_v3_small',
                     'crnn_mobilenet_v3_large',
                     'sar_resnet31',
                     'master',
                     'vitstr_small',
                     'vitstr_base',
                     'parseq',]

predictor = ocr_predictor(det_arch='fast_base', reco_arch='crnn_vgg16_bn', pretrained=True)


In [42]:
new_exhibit = ATC_amendment(new_orig_pdf_path, new_ocr_pdf_path, config_path, ':')
new_exhibit.get_text_lines_from_original()
new_exhibit.run_ocr(predictor)

new_exhibit.get_text_lines_from_ocr()
new_exhibit.get_section_bounds()


model # gps cma-b/6521/e0-6 atm1900d-1cwa radio 4478 b71 rru22 apxvaa24_43-u-
dimensions 12" x 9" x 6" 96" x 24" x 8.5" 81.1" x 7.7" x 4.8" 8.6" x 10" x 2.6" 15" x 13.2" x 7.4" 20.2" x 13.2" x 6.9"
azimuths/dir. of n/a 60/180/300 60/80/300 60/180/300 60/180/300 60
quant. per n/a 1/1/1 2/2/2 1/1/1 1/1/1 1
tx/rx frequency n/a mhz mhz n/a n/a mhz
tx frequency n/a 668-688 n/a n/a 1930-1945,2145- 2145-2155,1735-
rx frequency n/a 622-642 n/a n/a 1850-1865,1745- 1745-1755,2135-
using unlicensed no no no no no no
configuration qty: 1
dimensions 20.2" x 13.2" x 6.9" 6.23' x 6.23' x 4.32' 10' x -' x -' 9.1" x 9.2" x 3.9" 9.1" x 9.2" x 3.9" 4.23' x -' x -'
azimuths/dir. of 180/300 292.15 184.14 292.15 184.14 292.15
quant. per 1/1 1 1 4 2 1
tx/rx frequency mhz ghz ghz n/a n/a ghz
tx frequency 6 11 n/a n/a 6 2145-2155,1735-
rx frequency 6 1 n/a n/a 6 1745-1755,2135-
using unlicensed no no no no no no
exhibit a-4
ground space requirements
total lease area sq.ft: 216.00 primary contiguous lease area 

In [43]:
new_exhibit.fill_implicit_keys('ground space requirements', left_mult=2, right_mult=2)
new_exhibit.extract_table_data()
new_exhibit.extract_text_data()


total lease area sq.ft: 216.00
primary contiguous lease area - l: 12.00'
primary contiguous lease area - w: 18.00'
primary contiguous lease area - h: 10.00'
primary contiguous lease area - sq.ft: 216.00
concrete pad - l: 10.00'
concrete pad - w: 16.00'
concrete pad - h: n/a
concrete pad - sq.ft: 160.00
outside primary lease area - l: n/a
outside primary lease area - w: n/a
outside primary lease area - h: n/a
outside primary lease area - sq.ft: n/a
generator: n/a
fuel tank size (gal): n/a
fuel type: n/a
fuel tank setback (radius): n/a
power provided by: utility company direct
telco/interconnect: n/a
type: n/a
quantity: n/a
tx power (watts): n/a
erp power (watts): n/a


In [44]:
new_exhibit.align_line_configuration_data(group_key = 'line configuration'   , max_line_types=2)
new_exhibit.align_line_configuration_data(group_key = 'conduit configuration', max_line_types=2)

Qty: 3
Type: Coax
Diameter: 1/2" Coax
Azimuth/Sector: 1/1/1
Qty: 6
Type: Coax
Diameter: 1 5/8"Coax
Azimuth/Sector: 2/2/2
Qty: 1
Type: Fiber/Hybrid
Diameter: 1 5/8"(1.63"-41.3mm) Fiber
Azimuth/Sector: 1/0/0
Qty: 3
Type: Coax
Diameter: 1/4" Coax
Azimuth/Sector: 1/1/1
Qty: 3
Type: Hard Line
Diameter: 1/8" HardLine
Azimuth/Sector: 2/1
Qty: 4
Qty: 4
Type: Control Cable
Type: Fiber/Hybrid
Diameter: 0.31" (7.8mm) Cable
Diameter: 0.19" (4.8mm) Fiber
Azimuth/Sector: 4
Azimuth/Sector: 4
Qty: 2
Qty: 2
Type: Control Cable
Type: Fiber/Hybrid
Diameter: 0.31" (7.8mm) Cable
Diameter: 0.19" (4.8mm) Fiber
Azimuth/Sector: 2
Azimuth/Sector: 2
Qty: 1
Qty: 1
Type: 2" conduit
Type: 2" conduit
containing:-;
containing:-;
Azimuth/Sector: 1
Azimuth/Sector: 1


In [46]:
new_exhibit.sections['ground space requirements']['data']
new_exhibit.sections['backup power requirements']['data']
new_exhibit.sections['utility requirements']['data']
new_exhibit.sections['transmitter & receiver specifications']['data']
new_exhibit.sections['equipment specifications']['data'].iloc[9]

Type                                                                       Radio/ODU
Manufacturer                                                                 Ceragon
Model #                                                                        RFU-D
Dimensions HxWxD                                                  9.1" x 9.2" x 3.9"
Weight (lbs.)                                                                   14.3
Location                                                                       Tower
RAD Center AGL                                                                155.0'
Tip Height                                                                    155.4'
Base Height                                                                   154.6'
Mount Type                                                                Pole Mount
Quantity                                                                           4
Azimuths/Dir. of Radiation                                       

In [144]:
old_exhibit = ATC_amendment(old_orig_pdf_path, old_ocr_pdf_path, config_path, ':')
old_exhibit.get_text_lines_from_original()
old_exhibit.run_ocr(predictor)


old_exhibit.get_text_lines_from_ocr()

old_exhibit.get_section_bounds()

old_exhibit.fill_implicit_keys('ground space requirements', left_mult=2, right_mult=2)
old_exhibit.extract_table_data()


old_exhibit.extract_text_data()

equipment tip 158.1' 160.0' 155.4' 155.4' 127.1' 125.4'
equipment base 151.9' 150.0' 154.6' 154.6' 122.9' 124.6'
azimuths/dir. of 292.15 184.14 184.14 292.15 292.15 292.15
quant. per 1 1 2 1 1 1
tx/rx frequency mhz ghz n/a n/a mhz n/a
using unlicensed no no no no no no
line quant. per 1 n/a see config. summary n/a 1 n/a
exhibit a-3
nmmu4u4ub
ground space requirements
total lease area sq. ft: 216.00' primary contiquous lease area l:12.00' w:18.00' h:10.00 sq. ft: 216.00
concrete pad 10.00' 16.00' n/a 160.00
outside primary lease area nia nia n/a sq. ft: nia
backup power requirements
generator: nia capacity(kw): nia fuel tank size(gal): n/a fuel type: nia fuel tank setback/radius): n/a
utility requirements
power provided by: utility company direct
telcolinterconnect: nia
transmitter & receiver specifications
type: n/a quantity: n/a txi power(watts): n/a erp(watts): n/a
equipment specifications
type panel panel tta rru/rrh rru/rrh rru/rrh
manufacturer cellmax rfs rfs ericsson ericsson eri

In [145]:
old_exhibit.align_line_configuration_data(group_key = 'separated configuration', max_line_types=2, type_key = 'line')
# old_exhibit.align_line_configuration_data(group_key = 'separated configuration', max_line_types=2, type_key = 'conduit')

AttributeError: 'list' object has no attribute 'lower'

In [115]:
old_exhibit.sections['ground space requirements']['data']
old_exhibit.sections['backup power requirements']['data']
old_exhibit.sections['utility requirements']['data']
old_exhibit.sections['transmitter & receiver specifications']['data']
old_exhibit.sections['equipment specifications']['data'].iloc[10]

                                        data_not_found
Type                                           rru/rrh
Manufacturer                                  ericsson
Model #                                          rru22
Dimensions HxWxD                    20.2" x 13.2"x6.9"
Weight(lbs.)                                      52.9
Location                                data_not_found
RAD Center AGL                          data_not_found
Equipment Tip Height                             180.8
Equipment Base Height                           179.2"
Mount Type                                    side arm
Quantity                                data_not_found
Azimuths/Dir. of Radiation                     180/300
Quant. Per Azimuth/Sector                          1/1
TX/RX Frequency Units                   data_not_found
TX Frequency                      2145-2155,1735- 1740
RX Frequency                      1745-1755,2135- 2140
Using Unlicensed Frequencies?           data_not_found
Equipment 

In [122]:
old_exhibit.ocr_text

{'pages': [{'page_idx': 0,
   'dimensions': (1584, 1224),
   'orientation': {'value': None, 'confidence': None},
   'language': {'value': None, 'confidence': None},
   'blocks': [{'geometry': ((0.37741268382352944, 0.046875),
      (0.45955882352941174, 0.0615234375)),
     'objectness_score': 0.8322032988071442,
     'lines': [{'geometry': ((0.37741268382352944, 0.046875),
        (0.45955882352941174, 0.0615234375)),
       'objectness_score': 0.8322032988071442,
       'words': [{'value': 'Exhibit',
         'confidence': 0.9793364405632019,
         'geometry': ((0.37741268382352944, 0.0478515625),
          (0.43175551470588236, 0.060546875)),
         'objectness_score': 0.8313275575637817,
         'crop_orientation': {'value': 0, 'confidence': None}},
        {'value': 'A-3',
         'confidence': 0.9999669790267944,
         'geometry': ((0.42922794117647056, 0.046875),
          (0.45955882352941174, 0.0615234375)),
         'objectness_score': 0.8330790400505066,
         '